In [54]:
#install.packages("tidyverse")
#install.packages("anytime")
#install.packages("gganimate")
#install.packages("maps")

In [55]:
library(tidyverse) 
library(lubridate)
library(anytime)
library(gganimate)
library(maps)
library(scales)

In [56]:
# Cores para os plots
pinkColors <- c("#FA3C8A", "#000000", "#370117", "#6E022F", "#A50446", "#DD055E", "#F9217A", "#FA599B", "#FC90BC", "#FDC7DD")

In [57]:
df <- read.csv2("../teste_dados_ecommerce.csv")

In [58]:
head(df)

,InvoiceNo,StoreId,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>
1,536365,11,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/01/10 08:26,2.55,17850,United Kingdom
2,536365,9,71053,WHITE METAL LANTERN,6,12/01/10 08:26,3.39,17850,United Kingdom
3,536365,17,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/01/10 08:26,2.75,17850,United Kingdom
4,536365,15,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/01/10 08:26,3.39,17850,United Kingdom
5,536365,17,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/01/10 08:26,3.39,17850,United Kingdom
6,536365,2,22752,SET 7 BABUSHKA NESTING BOXES,2,12/01/10 08:26,7.65,17850,United Kingdom


In [59]:
df$UnitPrice <- as.numeric(df$UnitPrice)
df$StoreId <- as.character(df$StoreId)
# Existem 2 tipos de máscaras diferentes nos dados de datas, deixei de usar o as.POSIXct para usar o parse_date_time.
df$InvoiceDate <- parse_date_time(df$InvoiceDate, orders = c('mdy HM', 'mdY HM'))

df <- df  %>% mutate(PurchaseValue = Quantity * UnitPrice)

In [60]:
head(df)

,InvoiceNo,StoreId,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,PurchaseValue
,<chr>,<chr>,<chr>,<chr>,<int>,<dttm>,<dbl>,<int>,<chr>,<dbl>
1,536365,11,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
2,536365,9,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
3,536365,17,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
4,536365,15,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
5,536365,17,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
6,536365,2,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850,United Kingdom,15.30


In [61]:
df  %>% select(Description, Quantity,PurchaseValue ) %>%  arrange(PurchaseValue)%>% distinct(Description)  %>% head()
# Como existem valores em Quantity negativos e com descrições do tipo "AMAZON FEE", "POSTAGE", etc. Pode-se subenteder que os valores negativos representam 
# Perdas ou despesas. Para calcular o valor das lojas que tem o maior faturamento não será necessário levar em consideração esses valores negativos, porém para 
# calcular os lucros de cada loja será preciso fazer a conta lucro = Faturamento - Despesas. Para todos os efeitos estou usando um filtro onde  PurchaseValue > 0 para evitar problemas
# em summarizes

,Description
,<chr>
1,"PAPER CRAFT , LITTLE BIRDIE"
2,MEDIUM CERAMIC TOP STORAGE JAR
3,Manual
4,AMAZON FEE
5,Adjust bad debt
6,POSTAGE


In [62]:
# Optei por retirar da análise esses valores que aparentam não serem operações de compra.
df <- df  %>% filter(!StockCode %in% c("S","M", "m", "POST", "DOT", "BANK CHARGES", "AMAZONFEE", "B", "C2")) 

In [63]:
# Insight 1: 10 lojas com os maiores faturamentos em pedidos
# Insight 1.1: Volume total de items de venda para as 10 lojas com maior faturamento em pedidos
insight1 <- df  %>% 
    select(StoreId, Quantity, PurchaseValue )  %>% 
    filter( PurchaseValue > 0) %>%  
    group_by(StoreId)  %>%
    summarise(sumQuantity = sum(Quantity), sumPurchaseValue = sum(PurchaseValue))  %>%  
    arrange(desc(sumPurchaseValue))  %>%
    head(10)
insight1

StoreId,sumQuantity,sumPurchaseValue
<chr>,<int>,<dbl>
17,351747,675923.9
4,359441,633415.7
13,276017,516043.0
3,267048,509737.9
1,269494,508641.1
9,277164,504821.3
5,266498,504570.6
2,268052,502800.7
10,267934,498952.6


In [64]:
plotInsight1 <- insight1  %>% 
    ggplot(aes(x =  reorder(StoreId, sumPurchaseValue), y = sumPurchaseValue,fill = StoreId)) + 
    geom_bar(stat= "identity") +  
    geom_text(aes(label = label_number_si(accuracy=0.01, )(sumPurchaseValue)), hjust = -0.3,fill = "white", label.size = 0)+
    coord_flip()+ 
    theme_minimal()+ 
    scale_fill_manual(values = pinkColors)+
    theme(  plot.title = element_text(hjust = 0.5, size = 24),axis.text.y = element_text(size = 20, hjust = 1), axis.ticks.x = element_blank(), axis.text.x = element_blank(),panel.background = element_rect(fill = 'grey'),) +
    labs(title="Insight 1: 10 lojas com os maiores faturamentos em pedidos",x ="StoreId", y = "PurchaseValue",fill="none") +
    guides(fill="none")

Warning message:
"Ignoring unknown parameters: fill, label.size"


In [65]:
plotInsight1.1 <- insight1 %>% ggplot(aes(x =  reorder(StoreId, sumPurchaseValue), y = sumQuantity,fill = StoreId)) +
    geom_bar(stat= "identity") +
    geom_text(aes(label = label_number_si(accuracy=0.01, )(sumQuantity)), hjust = -0.3,fill = "white", label.size = 0)+
    coord_flip()+ 
    theme_minimal()+ 
    scale_fill_manual(values = pinkColors)+
    theme( plot.title = element_text(hjust = 0.5, size = 24), axis.text.y = element_text(size = 20, hjust = 1),  axis.ticks.x = element_blank(), axis.text.x = element_blank(), panel.background = element_rect(fill = 'grey'),) +
    labs(title="Insight 1.1: Volume total de items de venda para as 10 lojas com maior faturamento em pedidos \n ordenado por maior faturamento",x ="StoreId", y = "sumQuantity",fill="none") +
    guides(fill="none")

Warning message:
"Ignoring unknown parameters: fill, label.size"


In [66]:
insight2 <- df %>% 
    select(CustomerID,InvoiceDate,  PurchaseValue) %>% 
    filter(PurchaseValue > 0, !is.na(CustomerID)) %>% 
    group_by(CustomerID, monthYear = floor_date(InvoiceDate, "month"))  %>% 
    summarise(PurchaseValue = sum(PurchaseValue)) %>% 
    arrange(monthYear)
    
insight2$monthYear <- format(insight2$monthYear, "%b/%Y")

head(insight2)

`summarise()` has grouped output by 'CustomerID'. You can override using the
`.groups` argument.


CustomerID,monthYear,PurchaseValue
<int>,<chr>,<dbl>
12347,dez/2010,711.79
12348,dez/2010,652.80
12370,dez/2010,1748.02
12377,dez/2010,1001.52
12383,dez/2010,555.72
12386,dez/2010,258.90


In [67]:
# Quantidade de clientes
insight2  %>% select(CustomerID)  %>% distinct()  %>% nrow()

[1] 4334

In [68]:
# Divisão da soma do valor total de faturamento do mês pelo número total de clientes (4334) para chegar no Ticket médio mensal
# Insight 2: Ticket médio mensal dos pedidos
insight2 <- insight2   %>% 
    select(monthYear, PurchaseValue)  %>% 
    group_by(monthYear = factor(monthYear, levels = unique(monthYear)))  %>% 
    summarise(PurchaseValue = round(sum(PurchaseValue)/4334, 2))
insight2

Adding missing grouping variables: `CustomerID`


monthYear,PurchaseValue
<fct>,<dbl>
dez/2010,130.94
jan/2011,130.14
fev/2011,102.29
mar/2011,134.88
abr/2011,105.05
mai/2011,152.40
jun/2011,151.00
jul/2011,136.76
ago/2011,146.93


In [69]:
plotInsight2 <- insight2  %>% ggplot(aes(x= monthYear, y = PurchaseValue, group = 1)) +
    geom_line(color="#fb3d8a",size=1.2) +
    geom_point(color="#fb3d8a",size=2) +
    geom_text(aes(label = PurchaseValue), nudge_y = 7) +
    theme_minimal() +
    labs(title="Insight 2: Ticket médio mensal dos pedidos")+
    theme( plot.title = element_text(hjust = 0.5, size = 24),axis.text.y = element_text(size = 20, hjust = 1), axis.text.x = element_text(angle = 15, size = 20, hjust = 1),panel.background = element_rect(fill = 'grey'),)

In [70]:
 # Como existem faturas que não constam número de clientes, para responder o volume médio mensal de vendas de todas as lojas optei por fazer 2 pesquisas:
 
 # Uma pesquisa não levando em consideração as faturas que não estão relacionadas a clientes:
 insight2.1Customer <- df %>% 
    select(CustomerID,InvoiceDate,  PurchaseValue) %>% 
    filter(PurchaseValue > 0, !is.na(CustomerID)) %>% 
    group_by(CustomerID, monthYear = floor_date(InvoiceDate, "month"))  %>% 
    summarise(PurchaseValue = sum(PurchaseValue)) %>% 
    arrange(monthYear)
insight2.1Customer$monthYear <- format(insight2.1Customer$monthYear, "%b/%Y")

# E uma pesquisa levando em consideração todos os valores, inclusive as faturas que não estão relacionadas a um cliente:
insight2.1NoCustomer <- df %>% 
    select(CustomerID,InvoiceDate,  PurchaseValue) %>% 
    filter(PurchaseValue > 0) %>% 
    group_by(CustomerID, monthYear = floor_date(InvoiceDate, "month"))  %>% 
    summarise(PurchaseValue = sum(PurchaseValue)) %>% 
    arrange(monthYear)
insight2.1NoCustomer$monthYear <- format(insight2.1NoCustomer$monthYear, "%b/%Y")

`summarise()` has grouped output by 'CustomerID'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'CustomerID'. You can override using the
`.groups` argument.


In [71]:
insight2.1Customer <- insight2.1Customer  %>% 
    select(monthYear, PurchaseValue)  %>% 
    group_by(monthYear = factor(monthYear, levels = unique(monthYear)))  %>% 
    summarise(PurchaseValue = round(sum(PurchaseValue), 2))

# Para a primeira análise temos R$ 673.928,20 de volume médio mensal de vendas.
insight2.1Customer %>% select(PurchaseValue)  %>% summarise(mean(PurchaseValue))

insight2.1NoCustomer <- insight2.1NoCustomer  %>% 
    select(monthYear, PurchaseValue)  %>% 
    group_by(monthYear = factor(monthYear, levels = unique(monthYear)))  %>% 
    summarise(PurchaseValue = round(sum(PurchaseValue), 2))

# Para a segunda análise temos R$ 790.163 de volume médio mensal de vendas.
insight2.1NoCustomer %>% select(PurchaseValue)  %>% summarise(mean(PurchaseValue))

Adding missing grouping variables: `CustomerID`


mean(PurchaseValue)
<dbl>
673928.2


Adding missing grouping variables: `CustomerID`


mean(PurchaseValue)
<dbl>
790163


In [72]:
# Insight 3: Países com os maiores números de vendas.
insight3 <- df %>% select(Country,PurchaseValue)  %>% 
    filter( PurchaseValue > 0) %>% 
    group_by(Country) %>% 
    summarise(PurchaseValue = sum(PurchaseValue)) %>% 
    arrange(desc(PurchaseValue))

# Pequena mudança manual para plotar no mapa mundi.
insight3$Country[insight3$Country == "United Kingdom"] <- "UK"
insight3


Country,PurchaseValue
<chr>,<dbl>
UK,8748247.47
Netherlands,283889.34
EIRE,271164.30
Germany,205569.89
France,184582.74
Australia,138171.31
Spain,55725.11
Switzerland,53087.90
Japan,37416.37


In [73]:
world_map <- map_data("world")
world_map <- subset(world_map, region != "Antarctica")
# HeatMap dos países com os maiores números de vendas.
plotInsight3 <- ggplot(data = insight3) +
  geom_map(dat = world_map, map = world_map, aes(map_id = region),fill = "grey", color = "#7f7f7f", size = 0.1) +
  geom_map(map = world_map, aes(map_id = Country, fill = PurchaseValue), size = 0.25, color = "#7f7f7f") +
  scale_fill_gradient(low = "#ffffff", high = "#FA3C8A",trans = "log") +
  coord_fixed(1.3) +
  expand_limits(x = world_map$long, y = world_map$lat)+
  theme(  plot.title = element_text(hjust = 0.5, size = 24),axis.ticks.x = element_blank(),axis.text.x = element_blank(), axis.ticks.y = element_blank(),axis.text.y = element_blank())+
  labs(title="Insight 3: HeatMap dos países com maior volume de vendas",) 

In [74]:
# Insight 3.1: Relação de produtos mais vendidos dentro do país com maior volume de vendas (Reino Unido).
insight3.1 <- df %>% select(StockCode, Description,PurchaseValue, Country,Quantity)  %>% 
    filter( PurchaseValue > 0, Country == "United Kingdom") %>% 
    group_by(StockCode, Description,Country) %>% 
    summarise(PurchaseValue = sum(PurchaseValue), Quantity = sum(Quantity)) %>% 
    arrange(desc(PurchaseValue))  

insight3.1 %>% head(10)

`summarise()` has grouped output by 'StockCode', 'Description'. You can
override using the `.groups` argument.


StockCode,Description,Country,PurchaseValue,Quantity
<chr>,<chr>,<chr>,<dbl>,<int>
23843,"PAPER CRAFT , LITTLE BIRDIE",United Kingdom,168469.60,80995
22423,REGENCY CAKESTAND 3 TIER,United Kingdom,142273.29,11068
85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom,98545.24,35448
47566,PARTY BUNTING,United Kingdom,93658.53,16973
85099B,JUMBO BAG RED RETROSPOT,United Kingdom,86471.34,44264
23166,MEDIUM CERAMIC TOP STORAGE JAR,United Kingdom,80575.63,77036
22086,PAPER CHAIN KIT 50'S CHRISTMAS,United Kingdom,62742.54,18530
84879,ASSORTED COLOUR BIRD ORNAMENT,United Kingdom,54756.79,33735
79321,CHILLI LIGHTS,United Kingdom,53336.56,10154


In [ ]:
plotInsight3.1 <- ggplot(data = insight3.1, aes(x = reorder(Description, -PurchaseValue), y = PurchaseValue, fill = Description))+
    geom_bar(stat='identity')+
    theme( plot.title = element_text(hjust = 0.5, size = 24),axis.text.y = element_text(size = 20, hjust = 1), , axis.text.x = element_text(angle = 80, size = 20, hjust = 1),panel.background = element_rect(fill = 'grey'),) +
    scale_fill_manual(values = pinkColors)+
    labs(title="Insight 3.1: Os 10 Items mais vendidos no Reino Unido ",x ="Description", y = "PurchaseValue",fill="none") +
    guides(fill="none")

In [75]:

# Insight 3.2: Relação de produtos que tiveram os menores valores de compra dentro do país com maior volume de vendas (Reino Unido).

insight3.2 <- df %>% select(StockCode, Description,PurchaseValue, Country,Quantity)  %>% 
    filter( PurchaseValue > 0, Country == "United Kingdom") %>% 
    group_by(StockCode, Description,Country) %>% 
    summarise(PurchaseValue = sum(PurchaseValue), Quantity = sum(Quantity)) %>% 
    arrange(PurchaseValue) 
insight3.2 %>% head(10)

`summarise()` has grouped output by 'StockCode', 'Description'. You can
override using the `.groups` argument.


In [ ]:
plotInsight3.2 <- ggplot(data = insight3.2, aes(x = reorder(Description, PurchaseValue), y = PurchaseValue, fill = Description))+
    geom_bar(stat='identity')+
    #theme(axis.text.y = element_text(size = 20, hjust = 1), axis.text.x = element_text(angle = 80, size = 20, hjust = 1)) +
    theme( plot.title = element_text(hjust = 0.5, size = 24),axis.text.y = element_text(size = 20, hjust = 1), , axis.text.x = element_text(angle = 80, size = 20, hjust = 1),panel.background = element_rect(fill = 'grey'),) +
    scale_fill_manual(values = pinkColors)+
    labs(title="Insight 3.2: Os 10 Items com menor valor de compra no Reino Unido ",x ="Description", y = "PurchaseValue",fill="none") +
    guides(fill="none")

In [80]:
# Insight 3.3: Relação dos produtos menos comprados (Quantidade) dentro do país com maior volume de vendas (Reino Unido).
# OBS: Existem cerca de 86 produtos que só venderam 1 unidade.
insight3.3 <- df %>% select(StockCode, Description,PurchaseValue, Country,Quantity)  %>% 
    filter( PurchaseValue > 0, Country == "United Kingdom") %>% 
    group_by(StockCode, Description,Country) %>% 
    summarise(PurchaseValue = sum(PurchaseValue), Quantity = sum(Quantity)) %>% 
    arrange(Quantity) 
insight3.3 %>% head(10)

`summarise()` has grouped output by 'StockCode', 'Description'. You can
override using the `.groups` argument.


StockCode,Description,Country,PurchaseValue,Quantity
<chr>,<chr>,<chr>,<dbl>,<int>
20703,BLUE PADDED SOFT MOBILE,United Kingdom,4.25,1
20860,GOLD COSMETICS BAG WITH BUTTERFLY,United Kingdom,2.10,1
21009,ETCHED GLASS STAR TREE DECORATION,United Kingdom,1.25,1
21120,*Boombox Ipod Classic,United Kingdom,16.98,1
21160,KEEP OUT GIRLS DOOR HANGER,United Kingdom,3.36,1
21310,CAPIZ CHANDELIER,United Kingdom,29.95,1
21410,COUNTRY COTTAGE DOORSTOP GREEN,United Kingdom,8.47,1
21414,SCALLOP SHELL SOAP DISH,United Kingdom,2.10,1
21491,SET OF THREE VINTAGE GIFT WRAPS,United Kingdom,1.95,1


In [78]:
# Plot dos Insights em jpeg
jpeg(file = "insight1.jpeg", width=1280, height=720)
plotInsight1
dev.off()

jpeg(file = "insight1.1.jpeg", width=1280, height=720)
plotInsight1.1
dev.off()

jpeg(file = "insight2.jpeg", width=1280, height=720)
plotInsight2
dev.off()

jpeg(file = "insight3_map.jpeg", width=1280, height=720)
plotInsight3
dev.off()

jpeg(file = "insight3.1.jpeg", width=1280, height=900)
plotInsight3.1
dev.off()

jpeg(file = "insight3.2.jpeg", width=1280, height=900)
plotInsight3.2
dev.off()

# Salvando as alterações para serem usadas futuramente
write.csv(df,"../teste_dados_ecommerce_tratado.csv", row.names = FALSE)

png 
  2

png 
  2

png 
  2

png 
  2

ERROR while rich displaying an object: Error in `f()`:
! Insufficient values in manual scale. 3999 needed but only 10 provided.

Traceback:
1. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
2. tryCatchList(expr, classes, parentenv, handlers)
3. tryCatchOne(expr, names, parentenv, handlers[[1L]])
4. doTryCatch(return(expr), name, parentenv, handler)
5. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
6. repr::mime2repr[[mime]](obj)
7. repr_text.default

png 
  2

png 
  2